In [2]:
import os 

os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGSMITH_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "DemoLangGraph"

In [3]:
from langchain_groq import ChatGroq
llm = ChatGroq(model="gemma2-9b-it", groq_api_key=os.getenv("GROQ_API_KEY"))
llm

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x00000237E7D5A1D0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x00000237E7D5B640>, model_name='gemma2-9b-it', model_kwargs={}, groq_api_key=SecretStr('**********'))

### Start building Chatbot using LangGraph

In [4]:
from typing import Annotated 
from typing_extensions import TypedDict
from langgraph.graph import StateGraph, START, END 
from langgraph.graph.message import add_messages


In [5]:
class State(TypedDict):
    # Annotated is a way to add extra metadata to a type
    # add_messages is a special LangGraph reducer function that appends new messages
    # When a node returns new messages, it will be merged into the state
    # using this reducer function
    messages: Annotated[list, add_messages]

graph_builder = StateGraph(State)

In [6]:
def chatbot(state: State):
    return {
        "messages": llm.invoke(state["messages"])
    }

In [7]:
graph_builder.add_node("chatbot", chatbot)

In [8]:
graph_builder.add_edge(START, "chatbot")
graph_builder.add_edge("chatbot", END)

In [9]:
graph = graph_builder.compile()

In [11]:
while True:
    user_input = input("User: ")
    if (user_input.lower() in ["quit", "q"]):
        print("Exiting...")
        break
    for event in graph.stream({"messages": {"role": "user", "content": user_input}}):
        print(event.values())
        for value in event.values():
            print("Assistant: ", value["messages"].content)
    

dict_values([{'messages': AIMessage(content='Hi there! 👋\n\nHow can I help you today?\n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 15, 'prompt_tokens': 11, 'total_tokens': 26, 'completion_time': 0.027272727, 'prompt_time': 0.001867214, 'queue_time': 0.245825296, 'total_time': 0.029139941}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run--16238b19-a01d-4804-a4e3-5e9821a86f5a-0', usage_metadata={'input_tokens': 11, 'output_tokens': 15, 'total_tokens': 26})}])
Assistant:  Hi there! 👋

How can I help you today?

dict_values([{'messages': AIMessage(content='The capital of India is **New Delhi**. \n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 13, 'prompt_tokens': 16, 'total_tokens': 29, 'completion_time': 0.023636364, 'prompt_time': 0.001867904, 'queue_time': 0.246063306, 'total_time': 0.025504268}, 'model_name': 'gemma2-9b-it', 'system_fingerprint